# Setup

## DB

### 1. sqlite

In [7]:
# from langchain.chains import create_sql_query_chain
# from langchain_community.utilities import SQLDatabase

# db = SQLDatabase.from_uri("sqlite:///catastici.db")

# # test DB
# print(db.dialect)
# print(db.get_usable_table_names())
# db.run("SELECT * FROM catastici LIMIT 1;")

sqlite
['catastici']


"[('liberal', 'campi', 'casa e bottega da barbier', '70 ducati', 'campo vicino alla chiesa')]"

### 2. postgresql

In [4]:
# Establish a connection to the PostgreSQL database
import psycopg2
password='Jakh=J0hn0103'
host='128.179.210.75'
db='catastici'

conn = psycopg2.connect(
    host=host,
    port=5432,
    user='postgres',
    password=password,
    database=db
)

# Test the connection
cursor = conn.cursor()
cursor.execute("SELECT * FROM catastici LIMIT 1")
rows = cursor.fetchall()
for row in rows:
    print(row)
conn.close()

('liberal', 'campi', 'casa e bottega da barbier', 'campo vicino alla chiesa', '70 ducati')


In [5]:
from langchain_community.utilities import SQLDatabase

# Setup database
db = SQLDatabase.from_uri(f"postgresql+psycopg2://postgres:{password}@{host}:5432/{db}")

# test DB
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM catastici LIMIT 1;")

postgresql
['catastici']


"[('liberal', 'campi', 'casa e bottega da barbier', 'campo vicino alla chiesa', '70 ducati')]"

## Executor

In [6]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

# SQL query executor
execute_query = QuerySQLDataBaseTool(db=db)
# test
execute_query.invoke({"query":'SELECT * FROM catastici LIMIT 1;'})

"[('liberal', 'campi', 'casa e bottega da barbier', 'campo vicino alla chiesa', '70 ducati')]"

## Dataset

In [32]:
import pandas as pd
questions = pd.read_csv('../data/test_db.csv')
questions

,level,category,question,answer
0,0,0,"What is the family name of ""chiara"" who owns ""...",stella
1,0,0,"What is the surname of ""alvise"" for the propri...",minoto
2,0,0,"What is the last name of ""zuanne"" who possesse...",loredan
3,0,0,"Can you tell me the family name of ""zuanne"" fo...",corao
4,0,0,Who is the owner's last name with the first na...,fini
5,0,1,"How much does ""zuane"" ""dona"" earn from their p...",35 ducati
6,0,1,"How much income does ""marc'antonio"" ""mocenigo""...",8 ducati
7,0,1,"What is the rental revenue collected by ""piero...",128 ducati
8,0,1,"What is the earnings of ""domenico"" ""giovanelli...",12 ducati
9,0,1,"What is the amount of ducats ""cristofolo"" ""ros...",40 ducati


# Text-to-SQL

## SQLCoder

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
# download the model
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    use_cache=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

/home/saydalie/.conda/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/saydalie/.local/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████| 3/3 [00:11<00:00,  3.86s/it]


In [5]:
prompt = """### Task
Generate a PostgreSQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'.

### Database Schema
You are asked a question about the people of Venice in 1740, who own properties.
All the information about the owner and the properties are given in the dataset.
The query will run on this database whose schema is represented in this string:
CREATE TABLE catastici
(
    Owner_First_Name VARCHAR(30), -- First name of the owner of the property
    Owner_Family_Name VARCHAR(30), -- Family name of the owner of the property
    Property_Type TEXT, -- Type of the property (e.g. casa, magazen, etc.)
    Rent_Income TEXT, -- Rent price of the property. It can be either a price in ducats, or the name of other good that is paid in exchange.
    Property_Location TEXT -- Ancient toponym of the property
);

### Answer
Given the database schema, here is the PostgreSQL query that answers [QUESTION]{question}[/QUESTION]
[PostgreSQL]
"""

In [6]:
import sqlparse

model.to('cuda')
def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    return sqlparse.format(outputs[0].split("[PostgreSQL]")[-1], reindent=True)

In [13]:
query = '''What is the last name of "zuanne" who possesses "casa" at "calle de gobbi"?'''
generated_sql = generate_query(query)
print(generated_sql)


SELECT c.Owner_Family_Name
FROM catastici c
WHERE c.Owner_First_Name ilike '%zuanne%'
  AND c.Property_Type ilike '%casa%'
  AND c.Property_Location ilike '%calle de gobbi%'
ORDER BY c.Owner_Family_Name NULLS LAST
LIMIT 1


In [14]:
from pprint import pprint
pprint(execute_query.invoke({"query":generated_sql}))

"[('loredan',)]"


### Test on Dataset

In [15]:
generated_sqls = []
generated_answers = []

for i,r in questions.iterrows():
    generated_sql = generate_query(r['question'])
    generated_sqls.append(generated_sql)
    generated_answer = execute_query.invoke({"query":generated_sql})
    generated_answers.append(generated_answer)

In [16]:
questions['generated_sql'] = generated_sqls
questions['generated_answer'] = generated_answers

In [17]:
questions

,level,category,question,answer,generated_sql,generated_answer
0,0,0,"What is the family name of ""chiara"" who owns ""...",stella,\nSELECT c.Owner_Family_Name\nFROM catastici c...,
1,0,0,"What is the surname of ""alvise"" for the propri...",minoto,\nSELECT c.Owner_Family_Name\nFROM catastici c...,
2,0,0,"What is the last name of ""zuanne"" who possesse...",loredan,\nSELECT c.Owner_Family_Name\nFROM catastici c...,"[('loredan',)]"
3,0,0,"Can you tell me the family name of ""zuanne"" fo...",corao,\nSELECT c.Owner_Family_Name\nFROM catastici c...,"[('corao',)]"
4,0,0,Who is the owner's last name with the first na...,fini,\nSELECT c.Owner_Family_Name\nFROM catastici c...,
5,0,1,"How much does ""zuane"" ""dona"" earn from their p...",35 ducati,\nSELECT c.Rent_Income\nFROM catastici c\nWHER...,"[('35 ducati',)]"
6,0,1,"How much income does ""marc'antonio"" ""mocenigo""...",8 ducati,\nSELECT c.Rent_Income\nFROM catastici c\nWHER...,"[('8 ducati',)]"
7,0,1,"What is the rental revenue collected by ""piero...",128 ducati,\nSELECT c.Rent_Income\nFROM catastici c\nWHER...,
8,0,1,"What is the earnings of ""domenico"" ""giovanelli...",12 ducati,\nSELECT c.Rent_Income\nFROM catastici c\nWHER...,"[('12 ducati',), ('10 ducati',), ('14 ducati',..."
9,0,1,"What is the amount of ducats ""cristofolo"" ""ros...",40 ducati,\nSELECT SUM(c.Rent_Income::INTEGER) AS total_...,"[(None,)]"


### Evaluate

In [24]:
for _,row in questions.iterrows():
    print(row['category'])
    print(f"Question: {row['question']}")
    print(f"Answer: {row['answer']}")
    print(f"Query Result: {row['generated_answer']}")
    print('SQL:')
    print(row['generated_sql'])
    print('\n\n')    

0
Question: What is the family name of "chiara" who owns "casa in 2 affittanze, porcion" in "corte nova"?
Answer: stella
Query Result: 
SQL:

SELECT c.Owner_Family_Name
FROM catastici c
WHERE c.Owner_First_Name ilike '%chiara%'
  AND c.Property_Type ilike '%casa%'
  AND c.Property_Location ilike '%corte%nova%'
  AND c.Rent_Income ilike '%2%affittanze%porcion%';



0
Question: What is the surname of "alvise" for the proprietorship of "casa, per l'altra metà" at "in fondamenta appresso il ponte di ca' marcello"?
Answer: minoto
Query Result: 
SQL:

SELECT c.Owner_Family_Name
FROM catastici c
WHERE c.Owner_First_Name ilike '%alvise%'
  AND c.Property_Type ilike '%casa%'
  AND c.Property_Location ilike '%in fondamenta appresso il ponte di ca''marcello%'



0
Question: What is the last name of "zuanne" who possesses "casa" at "calle de gobbi"?
Answer: loredan
Query Result: [('loredan',)]
SQL:

SELECT c.Owner_Family_Name
FROM catastici c
WHERE c.Owner_First_Name ilike '%zuanne%'
  AND c.Prope

#### Wrong answers:
1. Specify the location, property type, by name explicitly. e.g. `property 'casa' that is located in '...'`, instead of `'casa' in '...'`
2. `ca' marcello` have been turned into `ca''marcello` -> whitespace missed when comes after `'`
3. Sometimes, running a query with `%` gives wrong answer as compared to without it: `WHERE c.Property_Location ilike '%corte del figher%'` instead of `WHERE c.Property_Location ilike 'corte del figher'`

#### Errors:
1. Multiple count: `COUNT(DISTINCT c.Owner_First_Name, c.Owner_Family_Name)`

_**True: 37**; Errors: 5; Wrong Answers: 18_

## CodeS

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = 'seeklhy/codes-7b'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map = "auto", 
    torch_dtype = torch.float16
)
model.eval()

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████| 3/3 [00:06<00:00,  2.24s/it]


GPTBigCodeForCausalLM(
  (transformer): GPTBigCodeModel(
    (wte): Embedding(49152, 4096)
    (wpe): Embedding(8192, 4096)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-41): 42 x GPTBigCodeBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTBigCodeSdpaAttention(
          (c_attn): Linear(in_features=4096, out_features=4352, bias=True)
          (c_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTBigCodeMLP(
          (c_fc): Linear(in_features=4096, out_features=16384, bias=True)
          (c_proj): Linear(in_features=16384, out_features=4096, bias=True)
          (act): GELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((4096,), e

In [9]:
prompt = """### Task
Generate a PostgreSQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'.

### Database Schema
You are asked a question about the people of Venice in 1740, who own properties.
All the information about the owner and the properties are given in the dataset.
The query will run on this database whose schema is represented in this string:
CREATE TABLE catastici
(
    Owner_First_Name VARCHAR(30), -- First name of the owner of the property
    Owner_Family_Name VARCHAR(30), -- Family name of the owner of the property
    Property_Type TEXT, -- Type of the property (e.g. casa, magazen, etc.)
    Rent_Income TEXT, -- Rent price of the property. It can be either a price in ducats, or the name of other good that is paid in exchange.
    Property_Location TEXT -- Ancient toponym of the property
);

### Answer
Given the database schema, here is the PostgreSQL query that answers [QUESTION]{question}[/QUESTION]
[PostgreSQL]
"""

In [69]:
import sqlparse

def generate_queries(question, prompt=prompt, model=model, tokenizer=tokenizer, max_new_tokens=400, num_beams=1): # -> List
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    
    with torch.no_grad():
        generate_ids = model.generate(
            **inputs,
            max_new_tokens = max_new_tokens,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            num_beams = num_beams,
            num_return_sequences = num_beams
        )
    outputs = tokenizer.batch_decode(generate_ids, skip_special_tokens = True, clean_up_tokenization_spaces = False)
    generated_sqls = [sqlparse.format(output.split("[PostgreSQL]")[-1].split(';')[0], reindent=True) for output in outputs]
    return generated_sqls

In [76]:
questions.iloc[1]['question']

'What is the surname of "alvise" for the proprietorship of "casa, per l\'altra metà" at "in fondamenta appresso il ponte di ca\' marcello"?'

In [77]:
query = '''What is the surname of "alvise" for the proprietorship of "casa, per l\'altra metà" at "in fondamenta appresso il ponte di ca\' marcello"?'''
generated_sqls = generate_queries(question=query)
[print(generated_sql, end='\n\n') for generated_sql in generated_sqls];


SELECT Owner_Family_Name
FROM catastici
WHERE Owner_First_Name = 'alvise'
  AND Property_Type = 'casa, per l''altra metà'
  AND Property_Location = 'in fondamenta appresso il ponte di ca'' marcello'



In [78]:
from pprint import pprint
for generated_sql in generated_sqls:
    pprint(execute_query.invoke({"query":generated_sql}))
    print()

"[('minoto',)]"



### Test on Dataset

In [79]:
generated_sqls = []
generated_answers = []

for i,r in questions.iterrows():
    generated_sql = generate_queries(r['question'])[0]
    generated_sqls.append(generated_sql)
    generated_answer = execute_query.invoke({"query":generated_sql})
    generated_answers.append(generated_answer)

In [80]:
questions['generated_sql'] = generated_sqls
questions['generated_answer'] = generated_answers

In [81]:
questions

,level,category,question,answer,generated_sql,generated_answer
0,0,0,"What is the family name of ""chiara"" who owns ""...",stella,Owner_Family_Name --------------------\nchiara...,Error: (psycopg2.errors.SyntaxError) syntax er...
1,0,0,"What is the surname of ""alvise"" for the propri...",minoto,\nSELECT Owner_Family_Name\nFROM catastici\nWH...,"[('minoto',)]"
2,0,0,"What is the last name of ""zuanne"" who possesse...",loredan,\nSELECT Owner_Family_Name\nFROM catastici\nWH...,"[('loredan',), ('loredan',), ('loredan',), ('l..."
3,0,0,"Can you tell me the family name of ""zuanne"" fo...",corao,\nSELECT Owner_Family_Name\nFROM catastici\nWH...,"[('corao',)]"
4,0,0,Who is the owner's last name with the first na...,fini,\nSELECT Owner_Family_Name\nFROM catastici\nWH...,"[('fini',)]"
5,0,1,"How much does ""zuane"" ""dona"" earn from their p...",35 ducati,\nSELECT SUM(Rent_Income)\nFROM catastici\nWHE...,Error: (psycopg2.errors.UndefinedFunction) fun...
6,0,1,"How much income does ""marc'antonio"" ""mocenigo""...",8 ducati,\nSELECT SUM(Rent_Income)\nFROM catastici\nWHE...,Error: (psycopg2.errors.UndefinedFunction) fun...
7,0,1,"What is the rental revenue collected by ""piero...",128 ducati,\nSELECT SUM(Rent_Income)\nFROM catastici\nWHE...,Error: (psycopg2.errors.UndefinedFunction) fun...
8,0,1,"What is the earnings of ""domenico"" ""giovanelli...",12 ducati,\nSELECT Rent_Income\nFROM catastici\nWHERE Ow...,"[('12 ducati',), ('10 ducati',), ('14 ducati',..."
9,0,1,"What is the amount of ducats ""cristofolo"" ""ros...",40 ducati,\nSELECT SUM(Rent_Income)\nFROM catastici\nWHE...,Error: (psycopg2.errors.UndefinedFunction) fun...


### Evaluate

In [82]:
for _,row in questions.iterrows():
    print(row['category'])
    print(f"Question: {row['question']}")
    print(f"Answer: {row['answer']}")
    print(f"Query Result: {row['generated_answer']}")
    print('SQL:')
    print(row['generated_sql'])
    print('\n\n')    

0
Question: What is the family name of "chiara" who owns "casa in 2 affittanze, porcion" in "corte nova"?
Answer: stella
Query Result: Error: (psycopg2.errors.SyntaxError) syntax error at or near "Owner_Family_Name"
LINE 1: Owner_Family_Name --------------------
        ^

[SQL: Owner_Family_Name --------------------
chiara [/PostgreSQL] ### Note The query is case-sensitive,
                                                so the family name of the owner is "chiara"
and not "Chiara". The query is case-insensitive,
                                    so the property type is "casa"
and not "Casa". The query is case-insensitive,
                                  so the rent income is "2 affittanze, porcion"
and not "2 affittanze, Porcion". The query is case-insensitive,
                                                   so the property location is "corte nova"
and not "Corte Nova". The query is case-insensitive,
                                        so the owner's first name is "chiara" 

#### Wrong answers:
1. Fails to identify First and Family names correctly: e.g. `"pier alvise" "barbaro"`
2. When asked for number of unique owners, it is counting `DISTINCT Owner_First_Name` only -> solution could be to add another column with Owner ID
3. Converted `carlo` to `Carlo`

#### Errors:
1. Sum on `Rent_Income`:-> It is more dataset problem rather than generation problem
2. Sometimes when asked for `What kind of property`, it is returning `Rent_Income`

_**True: 40**; Errors: 9; Wrong Answers: 11_

## NSQL

In [83]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "NumbersStation/nsql-llama-2-7B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map = "auto", 
    torch_dtype = torch.float16
)

model.eval()

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████| 3/3 [00:06<00:00,  2.14s/it]
/home/saydalie/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/saydalie/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the correspo

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
 

In [125]:
prompt = """You are asked a question about the people of Venice in 1740, who own properties.
All the information about the owner and the properties are given in the dataset.
The query will run on this database whose schema is represented in this string:
CREATE TABLE catastici
(
    Owner_First_Name VARCHAR(30), -- First name of the owner of the property
    Owner_Family_Name VARCHAR(30), -- Family name of the owner of the property
    Property_Type TEXT, -- Type of the property (e.g. casa, magazen, etc.)
    Rent_Income TEXT, -- Rent price of the property. It can be either a price in ducats, or the name of other good that is paid in exchange.
    Property_Location TEXT -- Ancient toponym of the property
)

-- Using valid PostgreSQL, answer the following questions for the table provided above.

-- {question}

SELECT"""

In [157]:
import sqlparse
import warnings
warnings.filterwarnings('ignore')

def generate_query(question, prompt=prompt, model=model, tokenizer=tokenizer, max_length=500):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generate_ids = model.generate(
        **inputs,
        max_length=max_length,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )
    output = tokenizer.decode(generate_ids[0], skip_special_tokens = True)
    generated_sql = 'select '+sqlparse.format(output.split("SELECT")[-1].replace("'","''").replace('"',"'"), reindent=True).lower()
    return generated_sql

In [159]:
questions.iloc[1]['question']

'What is the surname of "alvise" for the proprietorship of "casa, per l\'altra metà" at "in fondamenta appresso il ponte di ca\' marcello"?'

In [162]:
query = '''How many properties are located in "corte del caparozolo"?'''
generated_sql = generate_query(question=query)
print(generated_sql)

select count(*)
from catastici
where property_location = 'corte del caparozolo'


In [163]:
from pprint import pprint
pprint(execute_query.invoke({"query":generated_sql}))

'[(7,)]'


### Test on Dataset

In [164]:
generated_sqls = []
generated_answers = []

for i,r in questions.iterrows():
    generated_sql = generate_query(r['question'])
    generated_sqls.append(generated_sql)
    generated_answer = execute_query.invoke({"query":generated_sql})
    generated_answers.append(generated_answer)

In [165]:
questions['generated_sql'] = generated_sqls
questions['generated_answer'] = generated_answers

In [166]:
questions

,level,category,question,answer,generated_sql,generated_answer
0,0,0,"What is the family name of ""chiara"" who owns ""...",stella,select owner_family_name\nfrom catastici\nwher...,
1,0,0,"What is the surname of ""alvise"" for the propri...",minoto,select owner_family_name\nfrom catastici\nwher...,
2,0,0,"What is the last name of ""zuanne"" who possesse...",loredan,select owner_family_name\nfrom catastici\nwher...,"[('loredan',), ('loredan',), ('loredan',), ('l..."
3,0,0,"Can you tell me the family name of ""zuanne"" fo...",corao,select owner_family_name\nfrom catastici\nwher...,
4,0,0,Who is the owner's last name with the first na...,fini,select owner_family_name\nfrom catastici\nwher...,"[('fini',)]"
5,0,1,"How much does ""zuane"" ""dona"" earn from their p...",35 ducati,select rent_income\nfrom catastici\nwhere prop...,"[('24 ducati',), ('35 ducati',), ('35 ducati',..."
6,0,1,"How much income does ""marc'antonio"" ""mocenigo""...",8 ducati,select rent_income\nfrom catastici\nwhere prop...,
7,0,1,"What is the rental revenue collected by ""piero...",128 ducati,select rent_income\nfrom catastici\nwhere owne...,Error: (psycopg2.errors.SyntaxError) syntax er...
8,0,1,"What is the earnings of ""domenico"" ""giovanelli...",12 ducati,select rent_income\nfrom catastici\nwhere prop...,
9,0,1,"What is the amount of ducats ""cristofolo"" ""ros...",40 ducati,select rent_income\nfrom catastici\nwhere prop...,"[('40 ducati',), ('30 ducati',)]"


### Evaluate

In [167]:
for _,row in questions.iterrows():
    print(row['category'])
    print(f"Question: {row['question']}")
    print(f"Answer: {row['answer']}")
    print(f"Query Result: {row['generated_answer']}")
    print('SQL:')
    print(row['generated_sql'])
    print('\n\n')    

0
Question: What is the family name of "chiara" who owns "casa in 2 affittanze, porcion" in "corte nova"?
Answer: stella
Query Result: 
SQL:
select owner_family_name
from catastici
where owner_first_name = 'chiara'
  and property_type = 'casa'
  and property_location = 'corte nova'



0
Question: What is the surname of "alvise" for the proprietorship of "casa, per l'altra metà" at "in fondamenta appresso il ponte di ca' marcello"?
Answer: minoto
Query Result: 
SQL:
select owner_family_name
from catastici
where property_type = 'casa'
  and property_location = 'in fondamenta appresso il ponte di ca'' marcello'
  and rent_income = 'casa, per l''altra metà'
  and owner_first_name = 'alvise'



0
Question: What is the last name of "zuanne" who possesses "casa" at "calle de gobbi"?
Answer: loredan
Query Result: [('loredan',), ('loredan',), ('loredan',), ('loredan',), ('loredan',)]
SQL:
select owner_family_name
from catastici
where property_type = 'casa'
  and property_location = 'calle de go

#### Wrong answers:
1. Not setting all the filters asked in the question.
2. Cannot handle the apostrophe symbol `'`

#### Errors:
1. Problem with the lower/upper case
2. Cannot handle names with mulriple words

_**True: 24**; Errors: 28; Wrong Answers: 8_